# Wordcount

## Complete version

In [ ]:
import os

In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
from pycompss.api.task import task

In [ ]:
from pycompss.api.parameter import *

In [ ]:
if 'BINDER_SERVICE_HOST' in os.environ:
    ipycompss.start(graph=True, trace=True, debug=False,
                    project_xml='../xml/project.xml',
                    resources_xml='../xml/resources.xml')
else:
    ipycompss.start(graph=True, monitor=1000, trace=False, debug=False)

In [ ]:
@task(returns=list, file_path=FILE_IN)
def read_file(file_path):
    """ Read a file and return a list of words.
    :param file_path: file's path
    :return: list of words
    """
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data += line.split()
    return data

In [ ]:
@task(returns=dict)
def wordCount(data):
    """ Construct a frequency word dictorionary from a list of words.
    :param data: a list of words
    :return: a dictionary where key=word and value=#appearances
    """
    partialResult = {}
    for entry in data:
        if entry in partialResult:
            partialResult[entry] += 1
        else:
            partialResult[entry] = 1
    return partialResult

In [ ]:
#@task(returns=dict, priority=True)
@task(returns=dict)
def merge_two_dicts(dic1, dic2):
    """ Update a dictionary with another dictionary.
    :param dic1: first dictionary
    :param dic2: second dictionary
    :return: dic1+=dic2
    """
    for k in dic2:
        if k in dic1:
            dic1[k] += dic2[k]
        else:
            dic1[k] = dic2[k]
    return dic1

In [ ]:
from pycompss.api.api import compss_wait_on

# Get the dataset path
pathDataset = os.getcwd() + '/dataset'

# Read file's content execute a wordcount on each of them

result = {}
for fileName in os.listdir(pathDataset):
    file_path = os.path.join(pathDataset, fileName)
    data = read_file(file_path)
    result = merge_two_dicts(result,wordCount(data))

# Accumulate the partial results to get the final result.

# Wait for result
result = compss_wait_on(result)

In [ ]:
print("Result:")
from pprint import pprint
pprint(result)
print("Words: {}".format(sum(result.values())))

In [ ]:
ipycompss.stop()